In [1]:
from scipy import stats as ss

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm


In [2]:
filepath = 'https://raw.githubusercontent.com/data-8/textbook/main/assets/data/san_francisco_2019.csv'

sf2019 = pd.read_csv(filepath)

In [3]:
sf2019

In [4]:
sf2019.query("Job == 'Mayor'")

In [5]:
sf2019.sort_values('Total Compensation')

In [6]:
sf2019 = sf2019.query("Salary &gt; 15000")
sf2019

In [7]:
sf_bins = np.arange(0, 726000, 25000)
sf2019[['Total Compensation']].hist(bins=sf_bins)

In [8]:
sf2019.sort_values('Total Compensation', ascending=False).head(2)

In [9]:
pop_median = np.percentile(sf2019['Total Compensation'], 50)
pop_median

135747.0

In [10]:
our_sample = sf2019.sample(500, replace=False)
our_sample


In [11]:

our_sample[['Total Compensation']].hist(bins=sf_bins)

In [12]:
est_median = np.percentile(our_sample['Total Compensation'], 50)
est_median

138064.0

In [13]:
resample1 = our_sample.sample(frac=1, replace=True)

In [14]:
resample1[['Total Compensation']].hist(bins=sf_bins)

In [15]:
resampled_median_1 = np.percentile(resample1['Total Compensation'], 50)
resampled_median_1

135626.0

In [16]:
resample2 = our_sample.sample(frac=1, replace=True)
resampled_median_2 = np.percentile(resample2['Total Compensation'], 50)
resampled_median_2

135313.5

In [17]:
def one_bootstrap_median():
    resampled_table = our_sample.sample(frac=1, replace=True)
    bootstrapped_median = np.percentile(resampled_table['Total Compensation'], 50)
    return bootstrapped_median

In [22]:
one_bootstrap_median()

136454.5

In [23]:
num_repetitions = 5000
bstrap_medians = []
for i in np.arange(num_repetitions):
    bstrap_medians.append(one_bootstrap_median())

In [24]:
plt.hist(bstrap_medians)
plt.axvline(x=pop_median, color='r', label='pop median')
plt.legend()

In [25]:
left = np.percentile(bstrap_medians, 2.5)
left

133604.5

In [26]:
right = np.percentile(bstrap_medians, 97.5)
right

145382.0

In [27]:
plt.hist(bstrap_medians)
plt.axvline(x=pop_median, color='r', label='pop median')
plt.axvline(x=left, color='y')
plt.axvline(x=right, color='y')
plt.legend()

In [28]:
def bootstrap_median(original_sample, num_repetitions):
    medians = []
    for i in np.arange(num_repetitions):
        new_bstrap_sample = original_sample.sample(frac=1, replace=True)
        new_bstrap_median = np.percentile(new_bstrap_sample['Total Compensation'], 50)
        medians.append(new_bstrap_median)
    return medians

In [33]:
# THE BIG SIMULATION: This one takes several minutes.

# Generate 100 intervals and put the endpoints in the table intervals

left_ends = []
right_ends = []
original_sample = sf2019.sample(500, replace=False)

for i in tqdm(np.arange(100)):
    #original_sample = sf2019.sample(500, replace=False)
    medians = bootstrap_median(original_sample, 5000)
    left_ends.append(np.percentile(medians, 2.5))
    right_ends.append(np.percentile(medians, 97.5))

intervals = pd.DataFrame({'Left': left_ends, 'Right': right_ends })

100%|███████████████████| 100/100 [03:03&lt;00:00,  1.84s/it]


In [30]:
intervals

In [31]:
pop_median

135747.0

In [34]:
inside_interval = intervals.query("Left &lt; @pop_median and Right &gt; @pop_median")
inside_interval

Left       Right
0   132572.0000  142592.500
1   132572.0000  142159.000
2   132676.0000  142253.000
3   132659.2625  142166.075
4   132572.0000  142592.500
..          ...         ...
95  132795.0000  142592.500
96  132676.0000  143026.000
97  132705.3750  142592.500
98  132904.0000  142802.000
99  132676.0000  142592.500

[100 rows x 2 columns]